In [1]:
import scipy
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from src.IterativeFitting import IterativeFitting as IF
from src.CorrFuncs import covariance_matrix, trend_est

#### Example in GL Paper

In [2]:
x = np.array([2,6,11])
Nx = np.array([337,167,186,212])
M1x = 451
Lx = np.array([np.log(0.80),np.log(1.16),np.log(1.57)])
vx = np.array([0.0542,0.0563,0.0563])
A0 = M1x*Nx[1:]/(Nx.sum())

In [3]:
it_fit_ex = IF(Lx,A0,Nx,M1x)

In [4]:
A, B, a0, b0 = it_fit_ex.convexProgram()

In [5]:
A

array([ 70.33038633,  95.48567119, 124.67758726])

In [6]:
A_GL = it_fit_ex.GL()

[[0.03582153 0.01186944 0.01186944]
 [0.01186944 0.03337481 0.01186944]
 [0.01186944 0.01186944 0.03073736]] [-0.22314355  0.14842001  0.45107562]


In [7]:
A_GL

(array([ 70.20458464,  95.46963796, 124.85560373]),
 160.47017367350816,
 array([96.79541536, 90.53036204, 87.14439627]),
 176.52982632649184,
 2)

In [8]:
A_GLN = it_fit_ex.GL_linesearch()

In [9]:
A_GLN

(array([ 70.33753879,  95.44494158, 124.66704769]),
 160.55047193677308,
 array([96.66246121, 90.55505842, 87.33295231]),
 176.44952806322692,
 2)

In [32]:
trend_est(A,B,a0,b0,vx,x,Lx,unadj=False)

(0.045430215893878516, 0.0004268215544488761)

#### Example from alochol data in $\verb+dosresmeta+$

In [14]:
x = np.array([9.06,27.,45.,64.8])
Nx = np.array([414,261,228,44,34])
A0 = np.array([61,69,22,19])
M1x = np.sum(A0) + 126
Lx = np.array([-0.2231435,-0.0001,0.5306283,0.8754687])
vx = np.array([0.2233380,0.2337519,0.3765137,0.4440046])

In [15]:
alcohol_data = IF(Lx,A0,Nx,M1x)

In [16]:
result_alc = alcohol_data.convexProgram()

In [17]:
result_alc

(array([67.12135054, 68.86105977, 18.64972113, 17.32195872]),
 array([193.87864946, 159.13894023,  25.35027887,  16.67804128]),
 125.04590983884529,
 288.9540901611547)

#### RR example where everything works

In [ ]:
x = np.array([1.8286, 9.1992, 22.8571, 35.6667, 58.4257])
A = np.array([38, 43, 32, 16, 27])
a0 = 28
M1 = np.sum(A) + a0
B = np.array([43030.54, 53088.96, 45348.09, 19790.79, 19919.85])
b0 = 22185.73
L = np.array([-0.41673, -0.39555, -0.48842, -0.27903, 0.20232])
v = np.array([0.2510976, 0.2456013, 0.2633819, 0.3208115, 0.2861643])
v *= v